# PLANTILLA TRANSFORMACIÓN DE DATOS

## IMPORTAR PAQUETES

In [106]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import LabelEncoder
from category_encoders import TargetEncoder
from sklearn.preprocessing import KBinsDiscretizer
from sklearn.preprocessing import Binarizer
from sklearn.preprocessing import PowerTransformer
from sklearn.preprocessing import QuantileTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import MaxAbsScaler
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler

#Automcompletar rápido
%config IPCompleter.greedy=True

In [107]:
import fiser_tools
from fiser_tools import *

fiser_tools.misc.fis_light_theme()

## IMPORTAR LOS DATOS

1.- Sustituir la ruta del proyecto.

In [108]:
ruta_proyecto = '../..'

2.- Nombrar los ficheros de datos.

In [109]:
nombre_cat = 'cat_resultado_eda.pickle'
nombre_num = 'num_resultado_eda.pickle'

3.- Cargar los datos.

In [110]:
cat = pd.read_pickle(ruta_proyecto + '/02_Datos/03_Trabajo/' + nombre_cat).reset_index(drop = True)
num = pd.read_pickle(ruta_proyecto + '/02_Datos/03_Trabajo/' + nombre_num).reset_index(drop = True)

4.- Separar la target.

In [111]:
target = num[['contrata_fondos']].copy().reset_index(drop=True)

5.- Cargamos Excel de aplicación de transformaciones a las variables

In [112]:
transf_template = pd.read_excel("Transformaciones.xlsx",index_col=0,skiprows=1)
transf_template

OHE   OE   TE   DS   CS  BIN  YEO   QT  MMS   SS  \
VARIABLE                                                                       
estado_civil                  X  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN   
hipoteca                      X  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN   
impago                        X  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN   
consumo                       X  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN   
formacion                   NaN    X  NaN  NaN  NaN  NaN  NaN  NaN    X    X   
resultado_campana_anterior  NaN    X  NaN  NaN  NaN  NaN  NaN  NaN    X    X   
trabajo                     NaN  NaN    X  NaN  NaN  NaN  NaN  NaN    X    X   
mes                         NaN  NaN    X  NaN  NaN  NaN  NaN  NaN    X    X   
num_contactos_esta_campana  NaN  NaN  NaN    X    X  NaN  NaN  NaN    X    X   
num_dias_ultimo_contacto    NaN  NaN  NaN    X  NaN  NaN  NaN  NaN    X    X   
euribor3m                   NaN  NaN  NaN  NaN    X  NaN    X    X    X    X   
variacion_tasa_empleo       NaN  NaN  NaN  NaN  NaN    X  NaN  NaN    X    X   
edad                        NaN  NaN  NaN  NaN  NaN  NaN    X    X    X    X   

                             RS  
VARIABLE                         
estado_civil                NaN  
hipoteca                    NaN  
impago                      NaN  
consumo                     NaN  
formacion                     X  
resultado_campana_anterior    X  
trabajo                       X  
mes                           X  
num_contactos_esta_campana    X  
num_dias_ultimo_contacto      X  
euribor3m                     X  
variacion_tasa_empleo         X  
edad                          X

## TRANSFORMACIÓN DE CATEGÓRICAS

### One Hot Encoding

#### Variables a aplicar OHE

In [113]:
var_ohe = list(transf_template[transf_template['OHE']=='X'].index.values)
var_ohe

['estado_civil', 'hipoteca', 'impago', 'consumo']

#### Instanciar

In [114]:
ohe = OneHotEncoder(sparse = False, handle_unknown='ignore')

#### Entrenar y aplicar

In [115]:
cat_ohe = ohe.fit_transform(cat[var_ohe])

#### Guardar como dataframe

In [116]:
cat_ohe = pd.DataFrame(cat_ohe, columns = ohe.get_feature_names_out())

### Ordinal Encoding

#### Variables a aplicar OE

In [117]:
var_oe = list(transf_template[transf_template['OE']=='X'].index.values)
var_oe

['formacion', 'resultado_campana_anterior']

#### Orden de los valores de las variables

In [118]:
#Orden de la primera variable
orden_formacion = ['NULO','illiterate','basic.4y','basic.6y','basic.9y','high.school','professional.course','university.degree']

#Orden de la segunda variable
orden_resultado_campana_anterior = ['nonexistent','failure','success']

#### Instanciar

In [119]:
oe = OrdinalEncoder(categories = [orden_formacion,orden_resultado_campana_anterior],
                    handle_unknown = 'use_encoded_value',
                    unknown_value = 10)

#### Entrenar y aplicar

In [120]:
cat_oe = oe.fit_transform(cat[var_oe])

#### Guardar como dataframe

In [121]:
#Añadir sufijos a los nombres
nombres_oe = [variable + '_oe' for variable in var_oe]

#Guardar como dataframe
cat_oe = pd.DataFrame(cat_oe, columns = nombres_oe)

### Target Encoding

#### Variables a aplicar TE

In [122]:
var_te = list(transf_template[transf_template['TE']=='X'].index.values)
var_te

['trabajo', 'mes']

#### Instanciar

In [123]:
te = TargetEncoder(min_samples_leaf=100, return_df = False)

c:\Users\sgarciam\AppData\Local\Programs\Python\Python310\lib\site-packages\category_encoders\target_encoder.py:127: FutureWarning: Default parameter smoothing will change in version 2.6.See https://github.com/scikit-learn-contrib/category_encoders/issues/327
  warnings.warn("Default parameter smoothing will change in version 2.6."


#### Entrenar y aplicar

In [124]:
cat_te = te.fit_transform(cat[var_te], y = target)

#### Guardar como dataframe

In [125]:
#Añadir sufijos a los nombres
nombres_te = [variable + '_te' for variable in var_te]

#Guardar como dataframe
cat_te = pd.DataFrame(cat_te, columns = nombres_te)

### Fechas y textos

Estas variables requieren transformaciones más ad-hoc y no tan susceptibles de aplicar mediante una plantilla.

Acudir a su correspondientes notebooks para recordar los recursos para trabajarlas.

No obstante esta plantilla contiene esta sección para recordar que aquí sería el momento en el que se trabajarían.

## TRANSFORMACIÓN DE NUMÉRICAS

### Discretizar variables

#### Mediante distancia similar

##### Variables a discretizar mediante distancia similar

In [126]:
var_disc_ds = list(transf_template[transf_template['DS']=='X'].index.values)
var_disc_ds

['num_contactos_esta_campana', 'num_dias_ultimo_contacto']

##### Instanciar

In [127]:
disc_ds = KBinsDiscretizer(n_bins = 6, strategy = 'uniform', encode = 'ordinal')

##### Entrenar y aplicar

In [128]:
num_disc_ds = disc_ds.fit_transform(num[var_disc_ds])

##### Guardar como dataframe

In [129]:
#Añadir sufijos a los nombres
nombres_ds = [variable + '_disc_ds' for variable in var_disc_ds]

#Guardar como dataframe
num_disc_ds = pd.DataFrame(num_disc_ds,columns = nombres_ds)

#### Mediante número de casos similares

##### Variables a discretizar mediante número de casos similares

In [130]:
var_disc_cs = list(transf_template[transf_template['CS']=='X'].index.values)
var_disc_cs

['num_contactos_esta_campana', 'euribor3m']

##### Instanciar

In [131]:
disc_cs = KBinsDiscretizer(n_bins = 2, strategy = 'quantile', encode = 'ordinal')

##### Entrenar y aplicar

In [132]:
num_disc_cs = disc_cs.fit_transform(num[var_disc_cs])

##### Guardar como dataframe

In [133]:
#Añadir sufijos a los nombres
nombres_cs = [variable + '_disc_cs' for variable in var_disc_cs]

#Guardar como dataframe
num_disc_cs = pd.DataFrame(num_disc_cs,columns = nombres_cs)

### Binarizar variables

#### Variables a binarizar

In [134]:
var_bin = list(transf_template[transf_template['BIN']=='X'].index.values)
var_bin

['variacion_tasa_empleo']

#### Instanciar

In [135]:
bin = Binarizer(threshold=0)

#### Entrenar y aplicar

In [136]:
num_bin = bin.fit_transform(num[var_bin])

#### Guardar como dataframe

In [137]:
#Añadir sufijos a los nombres
nombres_bin = [variable + '_bin' for variable in var_bin]

#Guardar como dataframe
num_bin = pd.DataFrame(num_bin,columns = nombres_bin)

### Normalizar (Gauss)

#### Con Yeo-Johnson

##### Variables a normalizar con Yeo-Johnson

In [138]:
var_yeo = list(transf_template[transf_template['YEO']=='X'].index.values)
var_yeo

['euribor3m', 'edad']

##### Instanciar

In [139]:
yeo = PowerTransformer(method = 'yeo-johnson')

##### Entrenar y aplicar

In [140]:
num_yeo = yeo.fit_transform(num[var_yeo])

##### Guardar como dataframe

In [141]:
#Añadir sufijos a los nombres
nombres_yeo = [variable + '_yeo' for variable in var_yeo]

#Guardar como dataframe
num_yeo = pd.DataFrame(num_yeo,columns = nombres_yeo)

#### Con Quantile Transformer

##### Variables a normalizar con Quantile Transformer

In [142]:
var_qt = list(transf_template[transf_template['QT']=='X'].index.values)
var_qt

['euribor3m', 'edad']

##### Instanciar

In [143]:
qt = QuantileTransformer(output_distribution='normal')

##### Entrenar y aplicar

In [144]:
num_qt = qt.fit_transform(num[var_qt])

##### Guardar como dataframe

In [145]:
#Añadir sufijos a los nombres
nombres_qt = [variable + '_qt' for variable in var_qt]

#Guardar como dataframe
num_qt = pd.DataFrame(num_qt,columns = nombres_qt)

## UNIFICAR DATASETS TRANSFORMADOS

### Meter en una lista todos los dataframes generados

In [146]:
dataframes = []
dataframes.extend(value for name, value in locals().items() if name.startswith('cat_') or name.startswith('num_'))

### Unir todos los dataframes

In [147]:
df = pd.concat(dataframes, axis = 1)

## REESCALAR VARIABLES

### Con Min-Max

In [148]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19279 entries, 0 to 19278
Data columns (total 27 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   estado_civil_OTHERS                 19279 non-null  float64
 1   estado_civil_divorced               19279 non-null  float64
 2   estado_civil_married                19279 non-null  float64
 3   estado_civil_single                 19279 non-null  float64
 4   estado_civil_nan                    19279 non-null  float64
 5   hipoteca_no                         19279 non-null  float64
 6   hipoteca_unknown                    19279 non-null  float64
 7   hipoteca_yes                        19279 non-null  float64
 8   impago_no                           19279 non-null  float64
 9   impago_unknown                      19279 non-null  float64
 10  impago_yes                          19279 non-null  float64
 11  consumo_no                          19279

#### Variables a reescalar con Min-Max

In [149]:
mm = list(transf_template[transf_template['MMS']=='X'].index.values)
var_mms = []
for i in range(0,len(mm)):
    for col in df.filter(like = mm[i]).columns.values:
        var_mms.append(col)

#### Instanciar

In [150]:
mms = MinMaxScaler()

#### Entrenar y aplicar

In [151]:
df_mms = mms.fit_transform(df[var_mms])

#### Guardar como dataframe

In [152]:
#Añadir sufijos a los nombres
nombres_mms = [variable + '_mms' for variable in var_mms]

#Guardar como dataframe
df_mms = pd.DataFrame(df_mms,columns = nombres_mms)

### Con estandarización

#### Variables a reescalar con estandarización

In [153]:
ss = list(transf_template[transf_template['SS']=='X'].index.values)
var_ss  = []
for i in range(0,len(ss)):
    for col in df.filter(like = ss[i]).columns.values:
        var_ss.append(col)

#### Instanciar

In [154]:
ss = StandardScaler()

#### Entrenar y aplicar

In [155]:
df_ss = ss.fit_transform(df[var_ss])

#### Guardar como dataframe

In [156]:
#Añadir sufijos a los nombres
nombres_ss = [variable + '_ss' for variable in var_ss]

#Guardar como dataframe
df_ss = pd.DataFrame(df_ss,columns = nombres_ss)

### Con estandarización robusta

#### Variables a reescalar con estandarización robusta

In [157]:
rs = list(transf_template[transf_template['RS']=='X'].index.values)
var_rs  = []
for i in range(0,len(rs)):
    for col in df.filter(like = rs[i]).columns.values:
        var_rs.append(col)

#### Percentiles a usar para el rango

In [158]:
p_min = 10.0
p_max = 90.0

#### Instanciar

In [159]:
rs = RobustScaler(quantile_range=(p_min, p_max))

#### Entrenar y aplicar

In [160]:
df_rs = rs.fit_transform(df[var_rs])

#### Guardar como dataframe

In [161]:
#Añadir sufijos a los nombres
nombres_rs = [variable + '_rs' for variable in var_rs]

#Guardar como dataframe
df_rs = pd.DataFrame(df_rs,columns = nombres_rs)

## UNIFICAR DATASETS REESCALADOS

### Crear una lista con los dataframes a incluir en el tablón analítico

In [162]:
incluir = [df, df_mms, df_rs, df_ss, target]

### Unir todos los dataframes en el tablón analítico

In [163]:
df_tablon = pd.concat(incluir, axis = 1)

## GUARDAR DATASET TRAS TRANSFORMACIÓN DE DATOS

En formato pickle para no perder las modificaciones de metadatos.

In [164]:
#Definir los nombres del archivo
ruta_df_tablon = ruta_proyecto + '/02_Datos/03_Trabajo/' + 'df_tablon.pickle'

In [165]:
#Guardar los archivos
df_tablon.to_pickle(ruta_df_tablon)